## Towards quantum computing in the coupled basis

#### Imports

In [1]:
from typing import List
import numpy as np
from scipy.sparse import lil_matrix
from NSMFermions.nuclear_physics_utils import SingleParticleState,get_twobody_nuclearshell_model
from NSMFermions.hamiltonian_utils import FermiHubbardHamiltonian
from NSMFermions import QuasiParticlesConverterOnlynnpp,QuasiParticlesConverter
from scipy.sparse.linalg import eigsh

/home/ecosta/miniconda3/envs/annealing-nsm-env/lib/python3.10/site-packages/tqdm_joblib/__init__.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


#### Read the CKI basis

In [165]:
# ==============================================================
# CKI parser with: j-only representation, T and Tz expansion,
# single-particle energies stored per (j_a,j_b) couple.
# ==============================================================

tbme_dict = {}        # key: ((j_ap,j_bp),(j_a,j_b),J,T,Tz) -> V
states_set = set()    # (j_a,j_b,J,T,Tz)
couple_spe = {}       # (j_a,j_b) -> epsilon_a + epsilon_b

def decode_j(nlj):
    """Decode CKI nlj to j (last digit / 2)."""
    j2 = int(nlj) % 10
    return j2 / 2

def is_header(line):
    """Detect TBME header lines: 8 integers."""
    parts = line.split()
    if len(parts) != 8:
        return False
    try:
        [int(p) for p in parts]
        return True
    except ValueError:
        return False


with open("data/cki", "r") as f:
    lines = f.readlines()



# ==============================================================
# Step 2 — TBMEs with expansion on Tz
# ==============================================================

i = 0
while i < len(lines):
    line = lines[i].strip()
    if is_header(line):
        parts = line.split()
        T_min, T_max = int(parts[0]), int(parts[1])
        j_a, j_b = decode_j(parts[2]), decode_j(parts[3])
        j_ap, j_bp = decode_j(parts[4]), decode_j(parts[5])
        J_min, J_max = int(parts[6]), int(parts[7])

        T_list = range(T_min, T_max+1)
        J_list = range(J_min, J_max+1)

        # read all numeric values in block
        values = []
        i += 1
        while i < len(lines):
            l = lines[i].strip()
            if l == "" or is_header(l):
                break
            for num in l.split():
                try:
                    values.append(float(num))
                except ValueError:
                    pass
            i += 1

        # Assign values to all (J,T) and expand to Tz
        idx = 0
        for T in T_list:
            for J in J_list:
                if idx < len(values):
                    V = values[idx]
                    idx += 1
                    for Tz in range(-T, T+1):
                        key = ((j_ap, j_bp), (j_a, j_b), J, T, Tz)
                        tbme_dict[key] = V

                        # register states (if phase / antisymmetry condition requires)
                        if (-1)**(T + J) == -1:
                            states_set.add((j_a, j_b, J, T, Tz))
                            states_set.add((j_ap, j_bp, J, T, Tz))

        continue

    i += 1

# ==============================================================
# Step 3 — build ordered list and SPE alignment
# ==============================================================

states_list = sorted(states_set)
spe_vector = []

for (j_a, j_b, J, T, Tz) in states_list:
    if (j_a, j_b) in couple_spe:
        spe_vector.append(couple_spe[(j_a, j_b)])
    elif (j_b, j_a) in couple_spe:
        spe_vector.append(couple_spe[(j_b, j_a)])
    else:
        spe_vector.append(0.0)

# ==============================================================
# Output
# ==============================================================

print("Coupled basis states and SPE:")
for s, e in zip(states_list, spe_vector):
    print(f"{s} -> {e}")

print("\nFirst 10 TBME entries:")
for k, v in list(tbme_dict.items())[:]:
    print(k, v)
    
    

# ---------------------------------------------
# Extract single-particle energies per j
# -------------
# ---------------------------------------------
# Parse SINGLE-PARTICLE ENERGIES — first block only
# ---------------------------------------------
spe_per_j = {}

i = 0
spe_found = False
while i < len(lines) and not spe_found:
    parts = lines[i].split()

    # Look for the *first* orbital line: 4 integers
    if len(parts) >= 4 and all(p.isdigit() for p in parts[:4]):
        j1 = decode_j(parts[2])   # 103 -> 3/2
        j2 = decode_j(parts[3])   # 101 -> 1/2

        # Next line contains the two SPE values
        if i+1 < len(lines):
            energy_line = lines[i+1].split()
            if len(energy_line) >= 2:
                e1 = float(energy_line[0])
                e2 = float(energy_line[1])

                spe_per_j[j1] = e1
                spe_per_j[j2] = e2

                spe_found = True   # IMPORTANT: read only first block
                break
    i += 1

print("Single-particle energies extracted from first block:")
print(spe_per_j)
spe_vector = []
for (j_a, j_b, J, Tz, T) in states_list:
    E_pair = spe_per_j.get(j_a, 0.0) + spe_per_j.get(j_b, 0.0)
    spe_vector.append(E_pair)
    
print(spe_vector)

Coupled basis states and SPE:
(0.5, 0.5, 0, 1, -1) -> 0.0
(0.5, 0.5, 0, 1, 0) -> 0.0
(0.5, 0.5, 0, 1, 1) -> 0.0
(0.5, 0.5, 1, 0, 0) -> 0.0
(1.5, 0.5, 1, 0, 0) -> 0.0
(1.5, 0.5, 2, 1, -1) -> 0.0
(1.5, 0.5, 2, 1, 0) -> 0.0
(1.5, 0.5, 2, 1, 1) -> 0.0
(1.5, 1.5, 0, 1, -1) -> 0.0
(1.5, 1.5, 0, 1, 0) -> 0.0
(1.5, 1.5, 0, 1, 1) -> 0.0
(1.5, 1.5, 1, 0, 0) -> 0.0
(1.5, 1.5, 2, 1, -1) -> 0.0
(1.5, 1.5, 2, 1, 0) -> 0.0
(1.5, 1.5, 2, 1, 1) -> 0.0
(1.5, 1.5, 3, 0, 0) -> 0.0

First 10 TBME entries:
((1.5, 1.5), (1.5, 1.5), 0, 0, 0) 0.0
((1.5, 1.5), (1.5, 1.5), 1, 0, 0) -3.1398
((1.5, 1.5), (1.5, 1.5), 2, 0, 0) 0.0
((1.5, 1.5), (1.5, 1.5), 3, 0, 0) -6.6779
((1.5, 1.5), (1.5, 1.5), 0, 1, -1) -2.7352
((1.5, 1.5), (1.5, 1.5), 0, 1, 0) -2.7352
((1.5, 1.5), (1.5, 1.5), 0, 1, 1) -2.7352
((1.5, 1.5), (1.5, 1.5), 1, 1, -1) 0.0
((1.5, 1.5), (1.5, 1.5), 1, 1, 0) 0.0
((1.5, 1.5), (1.5, 1.5), 1, 1, 1) 0.0
((1.5, 1.5), (1.5, 1.5), 2, 1, -1) -0.649
((1.5, 1.5), (1.5, 1.5), 2, 1, 0) -0.649
((1.5, 1.5), (1.5, 1.5), 

#### Build up the many-body basis

Number of particles

In [156]:
# Desired total number of neutrons and protons
N_neutron = 2
N_proton = 0

In [196]:
import itertools

# Example: states_list = [(j_a,j_b,J,Tz), ...] from previous parser
# Let's take N = number of coupled states
N = len(states_list)
print(N)


def valid_bitstring(bitstring, states, N_neutron, N_proton):
    n_neutron = 0
    n_proton = 0
    for bit, state in zip(bitstring, states):
        if bit == 1:
            Tz = state[4]
            if Tz == -1:
                n_neutron += 2  # each pair contributes 2 particles of that type
            elif Tz == 1:
                n_proton += 2
            elif Tz == 0:
                n_neutron += 1
                n_proton += 1
    return n_neutron == N_neutron and n_proton == N_proton

# Generate all valid bitstrings (could be large!)
valid_configs = []
for bits in itertools.product([0,1], repeat=N):
    if valid_bitstring(bits, states_list, N_neutron, N_proton):
        valid_configs.append(bits)

print(f"Number of valid many-body states: {len(valid_configs)}")
print("Example bitstrings:")
for bits in valid_configs[:10]:
    print(bits)

16
Number of valid many-body states: 4
Example bitstrings:
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0)
(0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
(1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)


In [190]:
for base in valid_configs:
    idx=np.nonzero(base)[0][0]
    print(idx)
    print(states_list[idx])

12
(1.5, 1.5, 2, 1, -1)
8
(1.5, 1.5, 0, 1, -1)
5
(1.5, 0.5, 2, 1, -1)
0
(0.5, 0.5, 0, 1, -1)


In [221]:
def p_dag_p_matrix(a,b, basis):
    matrix=lil_matrix((len(basis), len(basis)), dtype=int)
    for idx, base in enumerate(basis):
        new_base=list(base)
        if a!=b:
            if base[a]==0 and base[b]==1:
                print(idx)
                new_base[a]=1
                new_base[b]=0
                
                jdx=basis.index(tuple(new_base))
                if jdx is not None:
                    matrix[jdx, idx]=1
        if a==b:
            if base[a]==1:
                matrix[idx,idx]=1
                
    return matrix

In [222]:
v_matrix=lil_matrix((len(valid_configs), len(valid_configs)), dtype=float)

for key, V in tbme_dict.items():
    (j1a,j1b), (j2a,j2b), J, T, Tz = key

    if (j1a,j1b,J,T,Tz) not in states_list:
        continue
    if (j2a,j2b,J,T,Tz) not in states_list:
        continue
    a=states_list.index((j1a,j1b,J,T,Tz))
    b=states_list.index((j2a,j2b,J,T,Tz))
    
    p_matrix=p_dag_p_matrix(a,b, valid_configs)
    if np.sum(p_matrix)!=0:
        print(a,b,p_matrix)
    v_matrix+=V*p_matrix

8 8 <List of Lists sparse matrix of dtype 'int64'
	with 1 stored elements and shape (4, 4)>
  Coords	Values
  (1, 1)	1
12 12 <List of Lists sparse matrix of dtype 'int64'
	with 1 stored elements and shape (4, 4)>
  Coords	Values
  (0, 0)	1
0
5 12 <List of Lists sparse matrix of dtype 'int64'
	with 1 stored elements and shape (4, 4)>
  Coords	Values
  (2, 0)	1
1
0 8 <List of Lists sparse matrix of dtype 'int64'
	with 1 stored elements and shape (4, 4)>
  Coords	Values
  (3, 1)	1
5 5 <List of Lists sparse matrix of dtype 'int64'
	with 1 stored elements and shape (4, 4)>
  Coords	Values
  (2, 2)	1
0 0 <List of Lists sparse matrix of dtype 'int64'
	with 1 stored elements and shape (4, 4)>
  Coords	Values
  (3, 3)	1


In [223]:
print(v_matrix.todense())

[[-0.649   0.      0.      0.    ]
 [ 0.     -2.7352  0.      0.    ]
 [-2.2055  0.     -1.1408  0.    ]
 [ 0.     -5.3229  0.      0.3397]]


In [224]:
h_spe_matrix=lil_matrix((len(valid_configs), len(valid_configs)), dtype=float)
for idx, base in enumerate(valid_configs):
    energy=0.0
    for bit, state in zip(base, states_list):
        if bit==1:
            j_a, j_b, J, T, Tz = state
            energy+=spe_per_j.get(j_a,0.0)+spe_per_j.get(j_b,0.0)
    h_spe_matrix[idx, idx]=energy
    
print(h_spe_matrix)

<List of Lists sparse matrix of dtype 'float64'
	with 4 stored elements and shape (4, 4)>
  Coords	Values
  (0, 0)	3.26
  (1, 1)	3.26
  (2, 2)	3.9
  (3, 3)	4.54


In [ ]:
h=0.5*(v_matrix+v_matrix.T)+h_spe_matrix
print(h)
e,psis=np.linalg.eigh(h.todense())

print(e)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 8 stored elements and shape (4, 4)>
  Coords	Values
  (0, 0)	3.26
  (0, 2)	1.10275
  (1, 1)	3.26
  (1, 3)	2.66145
  (2, 0)	-1.10275
  (2, 2)	3.9
  (3, 1)	-2.66145
  (3, 3)	4.54
[1.16268085 2.43175893 4.72824107 6.63731915]
